<a href="https://colab.research.google.com/github/ashmousaXX/Arabic-English-Translator/blob/main/Pattern_Final_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Task 1 (Rawan) : Preprocessing Dataset**

In [5]:
import re
import random
import os
import urllib.request

**Verify dataset exists**

In [6]:
dataset_path = "content/ara_.txt"

# Ensure the directory exists before downloading
if not os.path.exists(os.path.dirname(dataset_path)) and os.path.dirname(dataset_path) != '':
    os.makedirs(os.path.dirname(dataset_path), exist_ok=True)

if not os.path.exists(dataset_path):
    print("Downloading dataset from GitHub...")
    url = "https://raw.githubusercontent.com/SamirMoustafa/nmt-with-attention-for-ar-to-en/master/ara_.txt"
    urllib.request.urlretrieve(url, dataset_path)
    print(f"✓ Dataset downloaded successfully: {dataset_path}")

✓ Dataset downloaded successfully: content/ara_.txt


**Preview first 10 lines**

In [7]:
print("First 10 lines of the dataset:")
with open(dataset_path, "r", encoding="utf-8-sig") as f:
    lines = f.readlines()
    for i, line in enumerate(lines[:10]):
        print(f"Line {i+1}:", repr(line))

First 10 lines of the dataset:
Line 1: 'Hi.\tمرحبًا.\n'
Line 2: 'Run!\tاركض!\n'
Line 3: 'Help!\tالنجدة!\n'
Line 4: 'Jump!\tاقفز!\n'
Line 5: 'Stop!\tقف!\n'
Line 6: 'Go on.\tداوم.\n'
Line 7: 'Go on.\tاستمر.\n'
Line 8: 'Hello!\tمرحباً.\n'
Line 9: 'Hurry!\tتعجّل!\n'
Line 10: 'Hurry!\tاستعجل!\n'


**Data cleaning functions**

In [8]:
def normalize_arabic(text):
    text = str(text).strip()

    # Remove diacritics only
    text = re.sub(r"[\u0617-\u061A\u064B-\u0652]", "", text)

    # Normalize Alef variants ONLY
    text = re.sub("[إأآا]", "ا", text)

    # Remove non-Arabic characters
    text = re.sub(r"[^ء-ي0-9\s]", "", text)

    # Normalize spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text

def clean_english(text):
    text = str(text).strip().lower()
    text = re.sub(r"[^a-z0-9.,!?\'\" ]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

**Load and clean dataset**

In [9]:
MAX_AR_LEN = 80
MAX_EN_LEN = 80
MIN_LEN = 2

def load_dataset(path, separator="\t"):
    arabic_sentences = []
    english_sentences = []

    with open(path, "r", encoding="utf-8-sig") as f:
        for line in f:
            parts = line.strip().split(separator)
            if len(parts) != 2:
                continue

            en, ar = parts
            en = clean_english(en)
            ar = normalize_arabic(ar)

            if not en or not ar:
                continue

            # Length filtering (word-based)
            if not (MIN_LEN <= len(ar.split()) <= MAX_AR_LEN):
                continue
            if not (MIN_LEN <= len(en.split()) <= MAX_EN_LEN):
                continue

            arabic_sentences.append(ar)
            english_sentences.append(en)

    if len(arabic_sentences) == 0:
        raise ValueError("Dataset is empty or file format is wrong!")

    return arabic_sentences, english_sentences

def split_data(ar, en, train_ratio=0.8, val_ratio=0.1, seed=42):
    rnd = random.Random(seed)
    data = list(zip(ar, en))
    rnd.shuffle(data)
    ar, en = zip(*data)
    n = len(ar)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))
    return (
        ar[:train_end], en[:train_end],
        ar[train_end:val_end], en[train_end:val_end],
        ar[val_end:], en[val_end:]
    )

def save_to_file(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for line in data:
            f.write(line + "\n")

In [10]:
arabic, english = load_dataset(dataset_path)

print("Total sentences loaded:", len(arabic))
print("Max Arabic length:", max(len(s.split()) for s in arabic))
print("Max English length:", max(len(s.split()) for s in english))
print("Avg Arabic length:", sum(len(s.split()) for s in arabic) / len(arabic))
print("Avg English length:", sum(len(s.split()) for s in english) / len(english))

Total sentences loaded: 10600
Max Arabic length: 36
Max English length: 34
Avg Arabic length: 4.388301886792453
Avg English length: 5.759150943396226


**Run preprocessing**

In [11]:
train_ar, train_en, val_ar, val_en, test_ar, test_en = split_data(arabic, english)

print("Train:", len(train_ar))
print("Validation:", len(val_ar))
print("Test:", len(test_ar))
print("\nExample:")
print("Arabic:", train_ar[0])
print("English:", train_en[0])

Train: 8480
Validation: 1060
Test: 1060

Example:
Arabic: هو ترجم الاية الى اللغة الانكليزية
English: he translated the verse into english.


**Save processed files**

In [12]:
# Save processed files
save_to_file("train.ar", train_ar)
save_to_file("train.en", train_en)
save_to_file("val.ar", val_ar)
save_to_file("val.en", val_en)
save_to_file("test.ar", test_ar)
save_to_file("test.en", test_en)

# **Task 2 (Amal) : TOKENIZING**

In [13]:
import sentencepiece as spm
import os

print("Starting Tokenization...")

Starting Tokenization...


**Train SentencePiece Tokenizers (Arabic + English)**

In [14]:
def train_spm(input_file, model_prefix, vocab_size=6000):
    spm.SentencePieceTrainer.Train(
        f"--input={input_file} "
        f"--model_prefix={model_prefix} "
        f"--vocab_size={vocab_size} "
        f"--model_type=bpe "
        f"--pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3"
    )
    print(f" Tokenizer trained: {model_prefix}.model")

train_spm("train.ar", "arabic_tokenizer", vocab_size=6000)
train_spm("train.en", "english_tokenizer", vocab_size=6000)

 Tokenizer trained: arabic_tokenizer.model
 Tokenizer trained: english_tokenizer.model


In [15]:
# Load tokenizers
sp_ar = spm.SentencePieceProcessor()
sp_en = spm.SentencePieceProcessor()
sp_ar.load("arabic_tokenizer.model")
sp_en.load("english_tokenizer.model")

True

**Encode datasets to ID sequences**

In [16]:
def encode_file(input_path, output_path, sp):
    with open(input_path, "r", encoding="utf-8") as f_in, \
         open(output_path, "w", encoding="utf-8") as f_out:
        for line in f_in:
            ids = sp.encode(
                            line.strip(),
                            out_type=int,
                            add_bos=True,
                            add_eos=True
                           )

            f_out.write(" ".join(map(str, ids)) + "\n")

    print(f"Encoded: {output_path}")

In [17]:
# Encode Arabic
encode_file("train.ar", "train.ar.ids", sp_ar)
encode_file("val.ar", "val.ar.ids", sp_ar)
encode_file("test.ar", "test.ar.ids", sp_ar)

# Encode English
encode_file("train.en", "train.en.ids", sp_en)
encode_file("val.en", "val.en.ids", sp_en)
encode_file("test.en", "test.en.ids", sp_en)

Encoded: train.ar.ids
Encoded: val.ar.ids
Encoded: test.ar.ids
Encoded: train.en.ids
Encoded: val.en.ids
Encoded: test.en.ids


# **Task 3 Lamis : Transformer Model**



In [18]:
# ==================== COMPLETE TRANSFORMER MODEL ====================
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# ========== SPECIAL TOKENS ==========
PAD_ID = 0
UNK_ID = 1
BOS_ID = 2
EOS_ID = 3

# ========== POSITIONAL ENCODING ==========
def positional_encoding(position, d_model):
    angle_rads = np.arange(position)[:, np.newaxis] / np.power(
        10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / np.float32(d_model)
    )

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    return tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)

# ========== MASKING FUNCTIONS (Keras MultiHeadAttention-compatible) ==========

def create_padding_mask(seq):
    """
    Returns mask for keys: 1 for real tokens, 0 for PAD.
    Shape: (batch, 1, seq_len) -> will broadcast to (batch, query_len, seq_len)
    """
    mask = tf.cast(tf.not_equal(seq, PAD_ID), tf.float32)  # (B, S)
    return mask[:, tf.newaxis, :]                          # (B, 1, S)

def create_look_ahead_mask(size):
    """
    Causal mask: 1 where allowed, 0 where future is blocked.
    Shape: (size, size)
    """
    return tf.linalg.band_part(tf.ones((size, size), dtype=tf.float32), -1, 0)

def create_masks(inp, tar):
    """
    Returns:
      enc_self_mask: (B, 1, S_src)   broadcasted for encoder self-attn
      dec_self_mask: (B, T_tgt, T_tgt) for decoder self-attn (causal + padding)
      dec_cross_mask:(B, 1, S_src)   broadcasted for cross-attn keys (encoder output)
    """
    # Encoder self-attention mask (key padding only)
    enc_self_mask = create_padding_mask(inp)   # (B, 1, S_src)

    # Decoder cross-attention mask (mask encoder keys)
    dec_cross_mask = create_padding_mask(inp)  # (B, 1, S_src)

    # Decoder self-attention mask = causal * target_key_padding
    tgt_key_padding = create_padding_mask(tar) # (B, 1, T_tgt)
    T = tf.shape(tar)[1]
    causal = create_look_ahead_mask(T)         # (T_tgt, T_tgt)

    # Broadcast tgt padding to (B, T_tgt, T_tgt) and apply causal
    dec_self_mask = tgt_key_padding * causal[tf.newaxis, :, :]  # (B, T_tgt, T_tgt)

    return enc_self_mask, dec_self_mask, dec_cross_mask

# ========== FEED FORWARD NETWORK ==========
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        layers.Dense(dff, activation='relu'),
        layers.Dense(d_model)
    ])

# ========== ENCODER LAYER ==========
class EncoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super().__init__()
        self.mha = layers.MultiHeadAttention(
    num_heads=num_heads,
    key_dim=d_model // num_heads
)
        self.ffn = point_wise_feed_forward_network(d_model, dff)
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, x, training=False, mask=None):
      attn_output = self.mha(
          query=x, value=x, key=x,
          attention_mask=mask
      )
      attn_output = self.dropout1(attn_output, training=training)
      out1 = self.layernorm1(x + attn_output)

      ffn_output = self.ffn(out1)
      ffn_output = self.dropout2(ffn_output, training=training)
      out2 = self.layernorm2(out1 + ffn_output)
      return out2


# ========== ENCODER ==========
class Encoder(layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 input_vocab_size, maximum_position_encoding, rate=0.1):
        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.enc_layers = [
            EncoderLayer(d_model, num_heads, dff, rate)
            for _ in range(num_layers)
        ]

        self.dropout = layers.Dropout(rate)

    def call(self, x, training=False, mask=None):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x=x, training=training, mask=mask)

        return x

# ========== DECODER LAYER ==========
class DecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super().__init__()

        self.mha1 = layers.MultiHeadAttention(
    num_heads=num_heads,
    key_dim=d_model // num_heads
)
        self.mha2 = layers.MultiHeadAttention(
    num_heads=num_heads,
    key_dim=d_model // num_heads
)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, training=False, look_ahead_mask=None, padding_mask=None):
      # Self-attention (causal + target padding)
      attn1 = self.mha1(
          query=x, value=x, key=x,
          attention_mask=look_ahead_mask
      )
      attn1 = self.dropout1(attn1, training=training)
      out1 = self.layernorm1(attn1 + x)

      # Cross-attention (mask encoder padding on keys)
      attn2 = self.mha2(
          query=out1, value=enc_output, key=enc_output,
          attention_mask=padding_mask
      )
      attn2 = self.dropout2(attn2, training=training)
      out2 = self.layernorm2(attn2 + out1)

      ffn_output = self.ffn(out2)
      ffn_output = self.dropout3(ffn_output, training=training)
      out3 = self.layernorm3(ffn_output + out2)
      return out3

# ========== DECODER ==========
class Decoder(layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 target_vocab_size, maximum_position_encoding, rate=0.1):
        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [
            DecoderLayer(d_model, num_heads, dff, rate)
            for _ in range(num_layers)
        ]

        self.dropout = layers.Dropout(rate)

    def call(self, x, enc_output, training=False, look_ahead_mask=None, padding_mask=None):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.dec_layers[i](
                x=x,
                enc_output=enc_output,
                training=training,
                look_ahead_mask=look_ahead_mask,
                padding_mask=padding_mask
            )

        return x

# ========== COMPLETE TRANSFORMER ==========
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 input_vocab_size, target_vocab_size,
                 pe_input, pe_target, rate=0.1):
        super().__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                               input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                               target_vocab_size, pe_target, rate)

        self.final_layer = layers.Dense(target_vocab_size)

    def call(self, inp, tar, training=False, enc_padding_mask=None,
             look_ahead_mask=None, dec_padding_mask=None):

        enc_output = self.encoder(
            x=inp,
            training=training,
            mask=enc_padding_mask
        )

        dec_output = self.decoder(
            x=tar,
            enc_output=enc_output,
            training=training,
            look_ahead_mask=look_ahead_mask,
            padding_mask=dec_padding_mask
        )

        final_output = self.final_layer(dec_output)

        return final_output

# ========== MODEL HYPERPARAMETERS ==========
NUM_LAYERS = 4
D_MODEL = 256
DFF = 1024
NUM_HEADS = 8
DROPOUT_RATE = 0.1

Training

In [19]:
# ==================== COMPLETE TRAINING PIPELINE - FIXED ====================
import numpy as np
import tensorflow as tf
import time

# ========== 1. LOAD TOKENIZED DATA ==========
def load_ids_file(filename):
    data = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            tokens = [int(x) for x in line.strip().split()]
            if tokens:
                data.append(tokens)
    return data

train_src = load_ids_file("train.ar.ids")
train_tgt = load_ids_file("train.en.ids")
val_src = load_ids_file("val.ar.ids")
val_tgt = load_ids_file("val.en.ids")

print(f"Training samples: {len(train_src)}")
print(f"Validation samples: {len(val_src)}")

# ========== 2. CREATE DATASETS ==========
def create_tf_dataset(src_data, tgt_data, shuffle=True):
    def generator():
        for src, tgt in zip(src_data, tgt_data):
            yield (np.array(src, dtype=np.int32),
                   np.array(tgt, dtype=np.int32))

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(None,), dtype=tf.int32),
            tf.TensorSpec(shape=(None,), dtype=tf.int32)
        )
    )

    if shuffle:
        dataset = dataset.shuffle(10000)

    # 🔥 BUCKET BY SEQUENCE LENGTH (SOURCE LENGTH)
    dataset = dataset.apply(
        tf.data.experimental.bucket_by_sequence_length(
            element_length_func=lambda src, tgt: tf.shape(src)[0],
            bucket_boundaries=[10, 20, 30, 40, 50],
            bucket_batch_sizes=[64, 64, 32, 32, 16, 8],
            padded_shapes=([None], [None]),
            padding_values=(PAD_ID, PAD_ID),
            drop_remainder=False
        )
    )

    return dataset.prefetch(tf.data.AUTOTUNE)


train_dataset = create_tf_dataset(train_src, train_tgt, shuffle=True)
val_dataset = create_tf_dataset(val_src, val_tgt, shuffle=False)

# ========== 3. INITIALIZE MODEL ==========
input_vocab_size = sp_ar.get_piece_size()
target_vocab_size = sp_en.get_piece_size()

transformer = Transformer(
    num_layers=NUM_LAYERS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dff=DFF,
    input_vocab_size=input_vocab_size,
    target_vocab_size=target_vocab_size,
    pe_input=1000,
    pe_target=1000,
    rate=DROPOUT_RATE
)

# ========== 4. BUILD MODEL ==========
# Test with one batch to build model
for inp, tar in train_dataset.take(1):
    tar_input = tar[:, :-1]
    enc_mask, look_ahead_mask, dec_mask = create_masks(inp, tar_input)

    # FIX: Use keyword arguments here too!
    _ = transformer(
        inp=inp,
        tar=tar_input,
        training=False,
        enc_padding_mask=enc_mask,
        look_ahead_mask=look_ahead_mask,
        dec_padding_mask=dec_mask
    )
    break

print(f"Model built: {transformer.count_params():,} parameters")

# ========== 5. OPTIMIZER & LOSS ==========
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(tf.maximum(step, 1.0))
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.minimum(arg1, arg2)

learning_rate = CustomSchedule(D_MODEL)
optimizer = tf.keras.optimizers.Adam(
    learning_rate,
    beta_1=0.9,
    beta_2=0.98,
    epsilon=1e-9
)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none'
)

def loss_function(real, pred):
    mask = tf.math.not_equal(real, PAD_ID)
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_) / tf.maximum(tf.reduce_sum(mask), 1.0)

# ========== 6. TRAINING FUNCTIONS (FIXED) ==========
@tf.function(reduce_retracing=True)
def train_step(inp, tar):
    tar_input = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar_input)

    with tf.GradientTape() as tape:
        predictions = transformer(
            inp=inp,
            tar=tar_input,
            training=True,
            enc_padding_mask=enc_padding_mask,
            look_ahead_mask=look_ahead_mask,
            dec_padding_mask=dec_padding_mask
        )

        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    return loss

@tf.function(reduce_retracing=True)
def val_step(inp, tar):
    tar_input = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar_input)

    predictions = transformer(
        inp=inp,
        tar=tar_input,
        training=False,
        enc_padding_mask=enc_padding_mask,
        look_ahead_mask=look_ahead_mask,
        dec_padding_mask=dec_padding_mask
    )

    loss = loss_function(tar_real, predictions)

    return loss

# ========== 7. SIMPLE TRAINING LOOP ==========
def simple_train(epochs=5):
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        print("-" * 40)

        # Training
        train_loss = 0
        train_batches = 0

        for batch_idx, (inp_batch, tar_batch) in enumerate(train_dataset):
            batch_loss = train_step(inp_batch, tar_batch)
            train_loss += batch_loss
            train_batches += 1

            if batch_idx % 20 == 0:
                print(f"  Batch {batch_idx}: loss = {batch_loss.numpy():.4f}")

        avg_train = train_loss / train_batches
        print(f"  Train Loss: {avg_train.numpy():.4f}")

        # Validation
        val_loss = 0
        val_batches = 0

        for batch_idx, (inp_batch, tar_batch) in enumerate(val_dataset):
            batch_loss = val_step(inp_batch, tar_batch)
            val_loss += batch_loss
            val_batches += 1

        avg_val = val_loss / val_batches
        print(f"  Val Loss: {avg_val.numpy():.4f}")

        # Save
        if (epoch + 1) % 2 == 0:
            transformer.save_weights(f"transformer_epoch_{epoch+1}.weights.h5")
            print(f"  Model saved")

    transformer.save_weights("transformer_final.weights.h5")
    print("\nTraining complete!")

# ========== 8. START TRAINING ==========
print("\nStarting training...")
simple_train(epochs=40)

Training samples: 8480
Validation samples: 1060


Instructions for updating:
Use `tf.data.Dataset.bucket_by_sequence_length(...)`.


Model built: 11,986,800 parameters

Starting training...

Epoch 1/40
----------------------------------------
  Batch 0: loss = 8.6475
  Batch 20: loss = 8.5674
  Batch 40: loss = 8.3535
  Batch 60: loss = 8.0531
  Batch 80: loss = 7.7623
  Batch 100: loss = 7.5341
  Batch 120: loss = 7.2707
  Train Loss: 8.0028
  Val Loss: 7.2661

Epoch 2/40
----------------------------------------
  Batch 0: loss = 7.0685
  Batch 20: loss = 6.8610
  Batch 40: loss = 6.5680
  Batch 60: loss = 6.3239
  Batch 80: loss = 6.1022
  Batch 100: loss = 5.8753
  Batch 120: loss = 5.5792
  Train Loss: 6.3595
  Val Loss: 5.7608
  Model saved

Epoch 3/40
----------------------------------------
  Batch 0: loss = 5.3894
  Batch 20: loss = 5.7991
  Batch 40: loss = 5.0638
  Batch 60: loss = 4.6710
  Batch 80: loss = 4.5743
  Batch 100: loss = 4.4003
  Batch 120: loss = 4.5547
  Train Loss: 4.9258
  Val Loss: 4.9193

Epoch 4/40
----------------------------------------
  Batch 0: loss = 4.3720
  Batch 20: loss = 4.91

**Checking Point**





> Success indicators:

        Loss decreases gradually with each epoch
   
        Not all gradients are zero

        Predictions do not contain NaN or Infinity

        The model saves weights (.h5 files are available)








Check Loss function pattern

TEST TRANSLATION

In [20]:
# ========== TEST TRANSLATION IMMEDIATELY ==========
print("\n" + "="*60)
print("QUICK TRANSLATION TEST")
print("="*60)

def translate_greedy(sentence, max_len=40):
    """
    Translate an Arabic sentence to English using greedy decoding.

    Args:
        sentence (str): Arabic input sentence
        max_len (int): Maximum output length

    Returns:
        str: Translated English sentence
    """
    # 1. Normalize Arabic
    sentence = normalize_arabic(sentence)

    # 2. Encode source with BOS/EOS
    src_ids = sp_ar.encode(
        sentence,
        out_type=int,
        add_bos=True,
        add_eos=True
    )

    if len(src_ids) == 0:
        return ""

    inp = tf.constant([src_ids], dtype=tf.int32)

    # 3. Decoder starts with BOS
    decoded = tf.constant([[BOS_ID]], dtype=tf.int32)

    # 4. Autoregressive decoding
    for _ in range(max_len):
        enc_mask, dec_self_mask, dec_cross_mask = create_masks(inp, decoded)

        logits = transformer(
            inp=inp,
            tar=decoded,
            training=False,
            enc_padding_mask=enc_mask,
            look_ahead_mask=dec_self_mask,
            dec_padding_mask=dec_cross_mask
        )

        next_token = tf.argmax(logits[:, -1, :], axis=-1, output_type=tf.int32)
        next_id = int(next_token.numpy()[0])

        if next_id == EOS_ID:
            break

        decoded = tf.concat([decoded, [[next_id]]], axis=1)

    # 5. Remove BOS and decode
    output_ids = decoded.numpy()[0][1:]
    return sp_en.decode(output_ids.tolist())


# Test simple translations
import random

def load_text_file(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]

test_ar_text = load_text_file("test.ar")
test_en_text = load_text_file("test.en")  # optional
random.seed(42)
indices = random.sample(range(len(test_ar_text)), 10)
print("\n" + "="*60)
print("RANDOM TEST SET TRANSLATION (GREEDY)")
print("="*60)

for i in indices:
    ar = test_ar_text[i]
    ref = test_en_text[i] if i < len(test_en_text) else ""
    pred = translate_greedy(ar)

    print(f"\nAR:   {ar}")
    print(f"PRED: {pred}")
    print(f"REF:  {ref}")



QUICK TRANSLATION TEST

RANDOM TEST SET TRANSLATION (GREEDY)

AR:   توم ينظف اسنانه بالفرشاة ثلاث مرات على الاقل يوميا
PRED: tom spends a dentist appointment on his times.
REF:  tom brushes his teeth at least three times a day.

AR:   نصحته الا يضع الكثير من السكر
PRED: she advised him not to drive too much sugar.
REF:  she advised him not to use too much sugar.

AR:   ايمكنني الذهاب الى النهر
PRED: can i go to the river a river?
REF:  may i go to the river?

AR:   ما يحتاجه حقا هو وظيفة جيدة
PRED: what really needed is a good job.
REF:  what he needs most is a good job.

AR:   من الممكن انها لن تاتي غدا
PRED: she may never come here tomorrow.
REF:  she may not come here tomorrow.

AR:   اتى الولد راكضا
PRED: the boy began to cry.
REF:  the boy came running.

AR:   لعل ذلك صحيح
PRED: i guess is true.
REF:  perhaps that's true.

AR:   اعدك بذلك
PRED: i promise this.
REF:  i give you my word.

AR:   هل جننت
PRED: are you studying?
REF:  have you gone nuts?

AR:   لا يرغب توم في بيع مزرع

CHECK MODEL WEIGHTS

In [22]:
# ========== CHECK MODEL WEIGHTS ==========
print("\n" + "="*60)
print("MODEL WEIGHTS CHECK")
print("="*60)

# Check if weights are updated (not all zeros)
all_weights = transformer.get_weights()
zero_counts = 0
total_params = 0

for i, weight in enumerate(all_weights):
    total_params += np.prod(weight.shape)
    zero_elements = np.sum(weight == 0)
    zero_percentage = (zero_elements / np.prod(weight.shape)) * 100

    if zero_percentage > 95:  # إذا كان أكثر من 95% أصفار
        zero_counts += 1
        print(f"  ⚠ Layer {i}: {zero_percentage:.1f}% zeros")

print(f"\nModel Statistics:")
print(f"  Total parameters: {total_params:,}")
print(f"  Layers with >95% zeros: {zero_counts}/{len(all_weights)}")

if zero_counts > len(all_weights) / 2:
    print("  ❌ PROBLEM: Too many weights are zeros! Training didn't work.")
else:
    print("  ✅ Good: Weights are properly updated.")


MODEL WEIGHTS CHECK

Model Statistics:
  Total parameters: 11,986,800
  Layers with >95% zeros: 0/172
  ✅ Good: Weights are properly updated.


#With beam search


In [23]:
def beam_search_translate_fixed(sentence, beam_width=4, max_len=40, length_penalty=0.6):
    """
    Improved beam search translation with length normalization.
    """
    # 1. Normalize Arabic
    sentence = normalize_arabic(sentence)
    src_ids = sp_ar.encode(sentence, out_type=int, add_bos=True, add_eos=True)
    if len(src_ids) == 0:
        return []

    # 2. Encoder input
    encoder_input = tf.constant([src_ids], dtype=tf.int32)
    enc_padding_mask = create_padding_mask(encoder_input)
    encoder_output = transformer.encoder(encoder_input, training=False, mask=enc_padding_mask)

    # 3. Initialize beams
    beams = [{
        'sequence': [BOS_ID],
        'score': 0.0,
        'decoder_input': tf.constant([[BOS_ID]], dtype=tf.int32)
    }]

    for step in range(max_len):
        all_candidates = []

        for beam in beams:
            if beam['sequence'][-1] == EOS_ID:
                all_candidates.append(beam)
                continue

            # Decoder masks
            decoder_input = beam['decoder_input']
            look_ahead_mask = create_look_ahead_mask(tf.shape(decoder_input)[1])
            look_ahead_mask = tf.minimum(
                look_ahead_mask,
                create_padding_mask(decoder_input)[:, :, :tf.shape(decoder_input)[1]]
            )

            # Decoder output
            decoder_output = transformer.decoder(
                decoder_input,
                encoder_output,
                training=False,
                look_ahead_mask=look_ahead_mask,
                padding_mask=enc_padding_mask
            )
            logits = transformer.final_layer(decoder_output[:, -1, :])
            log_probs = tf.nn.log_softmax(logits).numpy()[0]

            # Top-k candidates
            top_k_ids = np.argsort(log_probs)[-beam_width:][::-1]
            for token_id in top_k_ids:
                seq = beam['sequence'] + [int(token_id)]
                score = beam['score'] + log_probs[token_id]

                # Length normalization
                norm_score = score / ((5 + len(seq)) ** length_penalty / (6 ** length_penalty))

                candidate = {
                    'sequence': seq,
                    'score': norm_score,
                    'decoder_input': tf.concat([beam['decoder_input'], tf.constant([[token_id]], dtype=tf.int32)], axis=1)
                }
                all_candidates.append(candidate)

        # Keep top beams
        all_candidates.sort(key=lambda x: x['score'], reverse=True)
        beams = all_candidates[:beam_width]

        # Stop if all beams ended with EOS
        if all(beam['sequence'][-1] == EOS_ID for beam in beams):
            break

    # Prepare final results
    results = []
    for beam in beams:
        output_ids = beam['sequence'][1:]  # remove BOS
        if output_ids and output_ids[-1] == EOS_ID:
            output_ids = output_ids[:-1]
        translation = sp_en.decode(output_ids)
        results.append({'translation': translation, 'score': beam['score'], 'sequence': beam['sequence']})

    return results





In [24]:
results = beam_search_translate_fixed("مرحبا كيف حالك", beam_width=5, length_penalty=1.0)
for i, r in enumerate(results):
    print(f"Beam {i+1} (score={np.exp(r['score']):.4f}): {r['translation']}")


Beam 1 (score=0.9320): hi. how are you?
Beam 2 (score=0.9192): hi! how are you?
Beam 3 (score=0.8417): hello, how are you?
Beam 4 (score=0.8347): welcome to how are you?
Beam 5 (score=0.5092): hello are you?


#BLEU SCORE EVALUATION


In [25]:

!pip install sacrebleu


In [26]:
import sacrebleu
from tqdm import tqdm

def evaluate_bleu(test_ar, test_en, decode_fn, max_samples=1000):

    predictions = []
    references = []

    for ar, ref in tqdm(list(zip(test_ar, test_en))[:max_samples]):
        pred = decode_fn(ar)
        predictions.append(pred)
        references.append(ref)

    bleu = sacrebleu.corpus_bleu(predictions, [references])
    return bleu


In [27]:
bleu_greedy = evaluate_bleu(
    test_ar_text,
    test_en_text,
    translate_greedy,
    max_samples=1000
)

print("🔹 BLEU (Greedy):")
print(bleu_greedy)


100%|██████████| 1000/1000 [24:54<00:00,  1.49s/it]

🔹 BLEU (Greedy):
BLEU = 25.45 54.1/30.0/19.5/13.2 (BP = 1.000 ratio = 1.028 hyp_len = 6998 ref_len = 6806)


In [28]:
def beam_best(sentence):
    results = beam_search_translate_fixed(sentence, beam_width=4)
    return results[0]["translation"]  # Best beam only

bleu_beam = evaluate_bleu(
    test_ar_text,
    test_en_text,
    beam_best,
    max_samples=1000
)

print("🔹 BLEU (Beam Search):")
print(bleu_beam)

100%|██████████| 1000/1000 [57:18<00:00,  3.44s/it]

🔹 BLEU (Beam Search):
BLEU = 24.44 51.7/29.0/18.9/12.6 (BP = 1.000 ratio = 1.109 hyp_len = 7546 ref_len = 6806)


# GUI

In [ ]:

import gradio as gr
import pandas as pd


history_data = []

def translate_text(source_text, source_lang, target_lang):
    """
    Translate text using the trained model
    """
    if not source_text.strip():
        return ""

    try:
        if source_lang == "Arabic" and target_lang == "English":

            translated = translate_greedy(source_text, max_len=40)

        else:
            translated = "⚠️ Invalid language selection"

        return translated

    except Exception as e:
        return f"❌ Error: {str(e)}"

def translate_and_save(source_text, source_lang, target_lang):
    """
    Translate text and add to history
    """
    if not source_text.strip():
        return "", create_history_df()

    # Get translation from model
    translated = translate_text(source_text, source_lang, target_lang)

    # Add to history
    history_entry = {
        "Source": source_lang,
        "Target": target_lang,
        "Source Text": source_text,
        "Translation": translated
    }
    history_data.insert(0, history_entry)

    return translated, create_history_df()

def create_history_df():
    """
    Create DataFrame from history for display
    """
    if not history_data:
        return pd.DataFrame(columns=["Source", "Target", "Source Text", "Translation"])
    return pd.DataFrame(history_data)

def clear_history():
    """
    Clear all translation history
    """
    global history_data
    history_data = []
    return create_history_df()


# Enhanced Custom CSS with animations
custom_css = """
@keyframes fadeInDown {
    from {
        opacity: 0;
        transform: translateY(-30px);
    }
    to {
        opacity: 1;
        transform: translateY(0);
    }
}

@keyframes fadeInUp {
    from {
        opacity: 0;
        transform: translateY(30px);
    }
    to {
        opacity: 1;
        transform: translateY(0);
    }
}

@keyframes slideInLeft {
    from {
        transform: translateX(-100px);
        opacity: 0;
    }
    to {
        transform: translateX(0);
        opacity: 1;
    }
}

@keyframes slideInRight {
    from {
        transform: translateX(100px);
        opacity: 0;
    }
    to {
        transform: translateX(0);
        opacity: 1;
    }
}

@keyframes bounceIn {
    0% {
        transform: scale(0.3);
        opacity: 0;
    }
    50% {
        transform: scale(1.05);
    }
    70% {
        transform: scale(0.9);
    }
    100% {
        transform: scale(1);
        opacity: 1;
    }
}

@keyframes pulse {
    0%, 100% {
        transform: scale(1);
        box-shadow: 0 0 0 0 rgba(102, 126, 234, 0.7);
    }
    50% {
        transform: scale(1.05);
        box-shadow: 0 0 0 10px rgba(102, 126, 234, 0);
    }
}

@keyframes gradient {
    0% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
    100% { background-position: 0% 50%; }
}

@keyframes shimmer {
    0% { left: -100%; }
    100% { left: 100%; }
}

@keyframes rotate {
    from { transform: rotate(0deg); }
    to { transform: rotate(360deg); }
}

@keyframes float {
    0%, 100% { transform: translateY(0px); }
    50% { transform: translateY(-10px); }
}

@keyframes glow {
    0%, 100% {
        box-shadow: 0 0 5px rgba(102, 126, 234, 0.5),
                    0 0 10px rgba(102, 126, 234, 0.3);
    }
    50% {
        box-shadow: 0 0 20px rgba(102, 126, 234, 0.8),
                    0 0 30px rgba(102, 126, 234, 0.5),
                    0 0 40px rgba(102, 126, 234, 0.3);
    }
}

@keyframes rainbow {
    0% { filter: hue-rotate(0deg); }
    100% { filter: hue-rotate(360deg); }
}

/* Container animations */
.gradio-container {
    animation: fadeInUp 1s ease-out;
    background: #FFB6C1 !important;
    background-size: 200% 200%;
}

/* Title animations */
h1 {
    animation: bounceIn 1s ease-out, float 3s ease-in-out infinite;
    background: linear-gradient(45deg, #667eea, #764ba2, #f093fb, #4facfe);
    background-size: 300% 300%;
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    background-clip: text;
    animation: gradient 5s ease infinite, bounceIn 1s ease-out;
    text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.1);
    text-align: center !important;
}

h3 {
    animation: slideInLeft 0.8s ease-out;
    position: relative;
}

h3::after {
    content: '';
    position: absolute;
    bottom: -5px;
    left: 0;
    width: 0;
    height: 3px;
    background: linear-gradient(90deg, #667eea, #764ba2);
    animation: expandWidth 1s ease-out forwards;
    animation-delay: 0.5s;
}

@keyframes expandWidth {
    to { width: 100%; }
}

/* Button mega animations */
button {
    transition: all 0.4s cubic-bezier(0.68, -0.55, 0.265, 1.55);
    position: relative;
    overflow: hidden;
}

button:hover {
    transform: translateY(-5px) scale(1.05);
    box-shadow: 0 10px 25px rgba(102, 126, 234, 0.5);
    animation: pulse 1s infinite;
}

button:active {
    transform: translateY(-2px) scale(1.02);
    animation: none;
}

button::before {
    content: '';
    position: absolute;
    top: 50%;
    left: 50%;
    width: 0;
    height: 0;
    border-radius: 50%;
    background: rgba(255, 255, 255, 0.5);
    transform: translate(-50%, -50%);
    transition: width 0.6s, height 0.6s;
}

button:hover::before {
    width: 400px;
    height: 400px;
}

button::after {

    position: absolute;
    top: 50%;
    left: 50%;
    transform: translate(-50%, -50%) scale(0);
    opacity: 0;
    transition: all 0.4s ease;
}

button:hover::after {
    transform: translate(-50%, -50%) scale(1);
    opacity: 1;
}

/* Primary button extra effects */
.primary {
    background: linear-gradient(45deg, #667eea 0%, #764ba2 50%, #f093fb 100%);
    background-size: 300% 300%;
    animation: gradient 4s ease infinite;
    border: none;
    color: white;
    font-weight: bold;
}

.primary:hover {
    animation: gradient 2s ease infinite, glow 2s ease infinite;
}

/* Textbox animations */
textarea, input[type="text"] {
    transition: all 0.4s ease;
    animation: fadeInUp 0.6s ease-out;
}

textarea:focus, input[type="text"]:focus {
    box-shadow: 0 0 0 4px rgba(102, 126, 234, 0.3),
                0 0 20px rgba(102, 126, 234, 0.4);
    transform: scale(1.02);
    animation: glow 2s ease infinite;
}

/* Dropdown animations */
.dropdown, select {
    transition: all 0.4s ease;
    animation: slideInRight 0.8s ease-out;
}

.dropdown:hover, select:hover {
    border-color: #667eea;
    transform: translateY(-2px);
    box-shadow: 0 5px 15px rgba(102, 126, 234, 0.3);
}

/* Label animations */
label {
    animation: fadeInDown 0.8s ease-out;
    transition: all 0.3s ease;
}

label:hover {
    color: #667eea;
    transform: translateX(5px);
}

/* Dataframe animations */
.dataframe {
    animation: fadeInUp 0.8s ease-out;
    transition: all 0.3s ease;
}

.dataframe:hover {
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.2);
    transform: translateY(-3px);
}

.dataframe tbody tr {
    animation: slideInLeft 0.5s ease-out;
    transition: all 0.3s ease;
}

.dataframe tbody tr:hover {
    background: linear-gradient(90deg, rgba(102, 126, 234, 0.1), rgba(118, 75, 162, 0.1));
    transform: translateX(5px);
}

.dataframe thead th {
    animation: bounceIn 0.8s ease-out;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
}

/* Column animations */
.col {
    animation: fadeInUp 0.8s ease-out;
}

.col:nth-child(1) {
    animation-delay: 0.1s;
}

.col:nth-child(2) {
    animation-delay: 0.3s;
}

/* Row animations */
.row {
    animation: fadeInUp 0.6s ease-out;
}

/* Shimmer effect for loading */
.pending::after {
    content: '';
    position: absolute;
    top: 0;
    left: -100%;
    width: 100%;
    height: 100%;
    background: linear-gradient(90deg,
        transparent,
        rgba(255, 255, 255, 0.4),
        transparent
    );
    animation: shimmer 2s infinite;
}

/* Add sparkle effect */
@keyframes sparkle {
    0%, 100% { opacity: 0; transform: scale(0); }
    50% { opacity: 1; transform: scale(1); }
}

/* Emoji animation in title */
h1::before {
    content: '🌐';
    display: inline-block;
    margin-right: 10px;
    animation: rotate 4s linear infinite, float 2s ease-in-out infinite;
}

/* Add border animation */
@keyframes borderGlow {
    0%, 100% { border-color: #667eea; }
    50% { border-color: #764ba2; }
}

textarea, input, select {
    animation: fadeInUp 0.6s ease-out, borderGlow 3s ease infinite;
}

/* Smooth page transitions */
* {
    transition: all 0.3s ease;
}

/* Add background pattern animation */
body {
    position: relative;
}

body::before {
    content: '';
    position: fixed;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background:
        radial-gradient(circle at 20% 50%, rgba(102, 126, 234, 0.1) 0%, transparent 50%),
        radial-gradient(circle at 80% 80%, rgba(118, 75, 162, 0.1) 0%, transparent 50%);
    animation: float 20s ease-in-out infinite;
    pointer-events: none;
    z-index: -1;
}
"""

# Create Gradio interface
with gr.Blocks(title="Our Translator", css=custom_css) as demo:
    gr.Markdown("# 🌐 Our Translator")
    gr.Markdown("Powered by Transformer Neural Machine Translation Model ")

    with gr.Row():
        with gr.Column(scale=2):
            # Translation Section
            gr.Markdown("### Translation")

            with gr.Row():
                source_lang = gr.Dropdown(
                    choices=["English", "Arabic"],
                    value="Arabic",
                    label="Source Language"
                )
                target_lang = gr.Dropdown(
                    choices=["English", "Arabic"],
                    value= "English",
                    label="Target Language"
                )

            source_text = gr.Textbox(
                label="Source Text",
                placeholder="Enter text to translate...",
                lines=5
            )

            translate_btn = gr.Button("Translate", variant="primary")

            translated_text = gr.Textbox(
                label="Translation",
                placeholder="Translation will appear here...",
                lines=5,
                interactive=False
            )

        with gr.Column(scale=1):
            # History Section
            gr.Markdown("### Translation History")
            clear_btn = gr.Button("Clear History", variant="secondary")

    # History Table (full width)
    with gr.Row():
        history_table = gr.Dataframe(
            value=create_history_df(),
            label="History",
            headers=[ "Source Text", "Translation"],
            interactive=False,
            wrap=True
        )

    # Event handlers
    translate_btn.click(
        fn=translate_and_save,
        inputs=[source_text, source_lang, target_lang],
        outputs=[translated_text, history_table]
    )

    clear_btn.click(
        fn=clear_history,
        inputs=[],
        outputs=[history_table]
    )

# Launch the app
demo.launch(share=True, debug=True)


/tmp/ipython-input-3669616885.py:430: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(title="Our Translator", css=custom_css) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://60f2281b66179a53bb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
